In [119]:
import json
import importlib
from nba_api.stats import endpoints


# Sample parameters for testing
SAMPLE_PARAMS = {
    'game_id': '0022300001',
    'player_id': '2544',  # LeBron James
    'team_id': '1610612747',  # Lakers
    'season': '2023-24',
    'season_type': 'Regular Season'
}


In [118]:
from nba_api.stats.endpoints import PlayerGameLogs
PlayerGameLogs(season_nullable='2023-24').get_data_frames()[0]

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,AVAILABLE_FLAG,MIN_SEC,TEAM_COUNT
0,2023-24,1628369.0,Jayson Tatum,Jayson,1610612738,BOS,Boston Celtics,0042300405,2024-06-17T00:00:00,BOS vs. DAL,...,1724,1166,3102,534,1,228,870,1.0,44:57,1
1,2023-24,1629029.0,Luka Dončić,Luka,1610612742,DAL,Dallas Mavericks,0042300405,2024-06-17T00:00:00,DAL @ BOS,...,1724,2016,50161,1777,1,228,1116,1.0,43:23,1
2,2023-24,1627759.0,Jaylen Brown,Jaylen,1610612738,BOS,Boston Celtics,0042300405,2024-06-17T00:00:00,BOS vs. DAL,...,1724,6225,2323,3967,4399,228,4581,1.0,44:15,1
3,2023-24,201950.0,Jrue Holiday,Jrue,1610612738,BOS,Boston Celtics,0042300405,2024-06-17T00:00:00,BOS vs. DAL,...,36799,13381,1990,9763,1,228,10123,1.0,43:08,1
4,2023-24,1628401.0,Derrick White,Derrick,1610612738,BOS,Boston Celtics,0042300405,2024-06-17T00:00:00,BOS vs. DAL,...,15154,14967,22798,12400,4399,228,10123,1.0,38:08,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52914,2023,1630137.0,Stephanie Watts,Stephanie,1611661324,MIN,Minnesota Lynx,1012300004,2023-05-05T00:00:00,MIN vs. WAS,...,36799,46755,31173,50955,4399,228,51040,2.0,1:20,1
52915,2023,1631033.0,Maya Dodson,Maya,1611661324,MIN,Minnesota Lynx,1012300004,2023-05-05T00:00:00,MIN vs. WAS,...,36799,46755,31173,50955,4399,228,51040,2.0,1:20,1
52916,2023,1641680.0,Myah Selland,Myah,1611661324,MIN,Minnesota Lynx,1012300004,2023-05-05T00:00:00,MIN vs. WAS,...,36799,46755,33082,50955,4399,228,51040,2.0,3:08,1
52917,2023,1641641.0,Robyn Parks,Robyn,1611661329,CHI,Chicago Sky,1012300005,2023-05-05T00:00:00,CHI @ DAL,...,36799,46755,34966,50955,4399,228,51040,2.0,1:46,1


In [ ]:
# Analysis: Find all endpoints whose priority is not 'None' and group by required parameters
import json
from collections import defaultdict

# Load the current endpoint configuration
with open('endpoint_priority_review.json', 'r') as f:
    config = json.load(f)

# Count endpoints by different criteria
total_endpoints = len(config['endpoints'])
priority_none = 0
priority_not_none = 0
latest_version_true = 0
latest_version_false = 0

# Analyze each endpoint and group by required params (only non-None priority AND no policy)
active_endpoints = []
active_endpoints_by_params = defaultdict(list)
no_policy_count = 0

for endpoint_name, endpoint_info in config['endpoints'].items():
    # Count priority status
    if endpoint_info.get('priority') == 'None':
        priority_none += 1
    else:
        priority_not_none += 1
        
        # Check if policy is not set (None, empty string, or missing)
        policy = endpoint_info.get('policy')
        has_no_policy = not policy or policy == 'None' or policy == ''
        
        if has_no_policy:
            no_policy_count += 1
            active_endpoints.append(endpoint_name)
            
            # Group by required parameters (only for non-None priority AND no policy endpoints)
            required_params = endpoint_info.get('required_params', [])
            param_key = tuple(sorted(required_params)) if required_params else ()
            active_endpoints_by_params[param_key].append(endpoint_name)
    
    # Count latest_version status
    if endpoint_info.get('latest_version') is True:
        latest_version_true += 1
    else:
        latest_version_false += 1

print("=== ACTIVE ENDPOINTS ANALYSIS ===")
print(f"Total endpoints: {total_endpoints}")
print()
print("PRIORITY STATUS:")
print(f"  priority = 'None':     {priority_none}")
print(f"  priority ≠ 'None':     {priority_not_none}")
print()
print("LATEST VERSION STATUS:")
print(f"  latest_version = True:  {latest_version_true}")
print(f"  latest_version = False: {latest_version_false}")
print()
print("🎯 ACTIVE ENDPOINTS (priority ≠ 'None' AND no policy set):")
print(f"  Count: {no_policy_count}")
print(f"  Percentage: {no_policy_count/total_endpoints*100:.1f}%")

print(f"\n📊 ACTIVE ENDPOINTS (NO POLICY) GROUPED BY REQUIRED PARAMETERS:")
print(f"Found {len(active_endpoints_by_params)} different parameter combinations")

# Sort groups by number of endpoints (most common first)
sorted_param_groups = sorted(active_endpoints_by_params.items(), key=lambda x: len(x[1]), reverse=True)

for param_tuple, endpoints in sorted_param_groups:
    if not param_tuple:
        param_str = "NO PARAMETERS"
    else:
        param_str = ", ".join(param_tuple)
    
    print(f"\n📋 REQUIRES: [{param_str}]")
    print(f"   Count: {len(endpoints)} endpoints")
    
    # Show all endpoints in this group
    for i, endpoint in enumerate(sorted(endpoints)):
        # Get priority and latest_version info
        endpoint_info = config['endpoints'][endpoint]
        priority = endpoint_info.get('priority', 'unknown')
        is_latest = endpoint_info.get('latest_version', False)
        latest_icon = "✅" if is_latest else "❌"
        policy = endpoint_info.get('policy', 'None')
        
        print(f"   {i+1:2d}. {endpoint} (priority: {priority}, policy: {policy}) {latest_icon}")

print(f"\n🎯 SUMMARY: {no_policy_count} active endpoints with no policy set - ready for policy assignment!")

=== ACTIVE ENDPOINTS ANALYSIS ===
Total endpoints: 136

PRIORITY STATUS:
  priority = 'None':     71
  priority ≠ 'None':     65

LATEST VERSION STATUS:
  latest_version = True:  125
  latest_version = False: 11

🎯 ACTIVE ENDPOINTS (priority ≠ 'None'):
  Count: 65
  Percentage: 47.8%

📊 ACTIVE ENDPOINTS GROUPED BY REQUIRED PARAMETERS:
Found 5 different parameter combinations

📋 REQUIRES: [team_id]
   Count: 10 endpoints
    1. CommonTeamRoster (priority: medium) ✅
    2. TeamDashLineups (priority: unknown) ✅
    3. TeamDashPtPass (priority: unknown) ✅
    4. TeamDashPtReb (priority: unknown) ✅
    5. TeamDashPtShots (priority: unknown) ✅
    6. TeamDashboardByGeneralSplits (priority: unknown) ✅
    7. TeamDashboardByShootingSplits (priority: unknown) ✅
    8. TeamPlayerDashboard (priority: unknown) ✅
    9. TeamPlayerOnOffDetails (priority: unknown) ✅
   10. TeamPlayerOnOffSummary (priority: unknown) ✅

📋 REQUIRES: [player_id]
   Count: 7 endpoints
    1. CommonPlayerInfo (priority: hi

In [429]:
# Update policy for all BoxScore endpoints
import json

# Load the current endpoint configuration
with open('endpoint_priority_review.json', 'r') as f:
    config = json.load(f)

# Find and update all BoxScore endpoints
boxscore_endpoints = []
updated_count = 0

for endpoint_name, endpoint_info in config['endpoints'].items():
    if 'BoxScore' in endpoint_name:
        boxscore_endpoints.append(endpoint_name)
        # Update policy to 'standard'
        if endpoint_info.get('policy') != 'standard':
            endpoint_info['policy'] = 'standard'
            updated_count += 1

print(f"Found {len(boxscore_endpoints)} BoxScore endpoints:")
for i, endpoint in enumerate(sorted(boxscore_endpoints)):
    print(f"  {i+1:2d}. {endpoint}")

print(f"\nUpdated policy to 'standard' for {updated_count} BoxScore endpoints")

# Save the updated configuration back to the file
with open('endpoint_priority_review.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Changes saved to endpoint_priority_review.json")

Found 21 BoxScore endpoints:
   1. BoxScoreAdvancedV2
   2. BoxScoreAdvancedV3
   3. BoxScoreDefensiveV2
   4. BoxScoreFourFactorsV2
   5. BoxScoreFourFactorsV3
   6. BoxScoreHustleV2
   7. BoxScoreMatchupsV3
   8. BoxScoreMiscV2
   9. BoxScoreMiscV3
  10. BoxScorePlayerTrackV2
  11. BoxScorePlayerTrackV3
  12. BoxScoreScoringV2
  13. BoxScoreScoringV3
  14. BoxScoreSimilarityScore
  15. BoxScoreSummaryV2
  16. BoxScoreTraditionalV2
  17. BoxScoreTraditionalV3
  18. BoxScoreUsageV2
  19. BoxScoreUsageV3
  20. GLAlumBoxScoreSimilarityScore
  21. HustleStatsBoxScore

Updated policy to 'standard' for 21 BoxScore endpoints
✅ Changes saved to endpoint_priority_review.json
